In [5]:
import pandas as pd

df = pd.read_csv("/home/squirrel/ccg-new/data/persp/db-102518-after-batch6/norm_persp_db_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46049 entries, 0 to 46048
Data columns (total 6 columns):
id                46049 non-null int64
author            46049 non-null object
claim_id          46049 non-null int64
perspective_id    46049 non-null int64
rel               46049 non-null object
comment           0 non-null float64
dtypes: float64(1), int64(3), object(2)
memory usage: 2.1+ MB


In [6]:
# Drop GOLD and TEST annotations
df = df[df.author != "GOLD"]
df.info()
df = df[df.author != "TEST"]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37914 entries, 4136 to 46048
Data columns (total 6 columns):
id                37914 non-null int64
author            37914 non-null object
claim_id          37914 non-null int64
perspective_id    37914 non-null int64
rel               37914 non-null object
comment           0 non-null float64
dtypes: float64(1), int64(3), object(2)
memory usage: 2.0+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 37880 entries, 4170 to 46048
Data columns (total 6 columns):
id                37880 non-null int64
author            37880 non-null object
claim_id          37880 non-null int64
perspective_id    37880 non-null int64
rel               37880 non-null object
comment           0 non-null float64
dtypes: float64(1), int64(3), object(2)
memory usage: 2.0+ MB


In [7]:
# Label Distributions
print(df.groupby('rel').count())

        id  author  claim_id  perspective_id  comment
rel                                                  
N     6265    6265      6265            6265        0
S    16254   16254     16254           16254        0
U    15361   15361     15361           15361        0


In [8]:
import numpy as np
def fleiss_kappa_1d(arr):
    """
    Easy version of fleiss kappa, where there is only one example
    """
    
    num_rater = np.sum(arr)
#     print(num_rater)
    p_e = np.sum(np.square(np.divide(arr, num_rater)))
#     print(p_e)
    p_o = 1/(num_rater * (num_rater - 1)) * (np.sum(np.square(arr)) - num_rater)
#     print(p_o)
    if p_e == 1: 
        return 1
    else:
        return (p_o - p_e) / (1 - p_e)


In [9]:
# IAA analysis -- Macro Avg Fleiss' Kappa among claims 
# Using NLTK implementation 
# https://www.nltk.org/_modules/nltk/metrics/agreement.html
from nltk.metrics.agreement import AnnotationTask
import numpy as np

claims = df.claim_id.unique()

# not_complete_author = {
# "A1JRMSNROZ1JX5",
# "A2FARB7LA24AU8",
# "AWZR4H0QX23W4",
# "A149ROBL26JWPJ",
# "AWJUGWPCUGKEG",
# "A3U74S280Z4B0",
# "AUCHGHY1IKZZK",
# "A2SWQM5X54P1O5",
# "A1OFSWG4TA4VVR",
# "A1DNJ17PE2RYJZ",
# "A2541C8MY0BYV3",
# "A3QSFE6GKO157S",
# "A95RHRMLHT89M",
# "A1ZD8RU6YB0VEU",
# "AU2RMH9IZP60M",
# "A3C5G8LGIAW0XL",
# "AVI7K876BV3QL",
# "A3C70KUK27HI3Y",
# "A2TBXASXZIRNNW",
# "A14HW8I4RYHNC4",
# "A3RTXKL0EGLRDJ",
# "A52FUUEDGSZT9",
# "A2YR5M2I1YCH75",
# "A2TZA6QIDOHF0R",
# "A3TLI95L5GOBKG",
# "AROOCBM042SJD",
# "A2P065E9CYMYJL",
# "A2E5PZUD6W4OA1",
# "A2E5PZUD6W4OA1",
# "A1ZT30BGR3266K",
# "A3GKCB2D8O2ICZ",
# "A2C73Y1COWCA51",
# "A3RYI5HXC2MJLN",
# "A1YTNGH5SMM2CJ",
# "ALJMVJ1L3NUTE",
# "AQYYBOOF0846O",
# "A2B6WQG0A9CKXQ",
# "ALEJV7D94ZLHF",
# "A5J0OW727ZCWY",
# }

arr_al = []
_data = []

for cid in claims:
    c_df = df[df.claim_id == cid]
    
    persps = c_df.perspective_id.unique()
    
    for pid in persps:
        p_df = c_df[c_df.perspective_id == pid]
        
        group = p_df.rel.value_counts()
        
        _s = _u = _n = 0
        if "S" in group.index:
            _s = group.S
        if "U" in group.index:
            _u = group.U
        if "N" in group.index:
            _n = group.N
        
        _data.append([cid, pid, _s, _u, _n])

print(len(_data))
with open("/home/squirrel/ccg-new/data/persp/db-102518-after-batch6/norm_persp_annotations.csv", 'w+') as fout:
    fout.write("claim,perspective,sup,und,ns\n")
    for entry in _data:
        _entry = [str(x) for x in entry]
        fout.write(",".join(_entry)+'\n')

    
# for entry in _data
    
#     triples = []
#     for idx, row in c_df.iterrows():
#         triples.append((row.author, row.perspective_id, row.rel))
    
#     an = AnnotationTask(data=triples)
#     k = an.multi_kappa()
#     arr_al.append(k)

# print("Avg IAA:{}".format(np.mean(arr_al)))
# print("Stddev:{}".format(np.std(arr_al)))

8135


In [62]:
import pandas as pd

iaa = "/home/squirrel/ccg-new/data/persp/db-101218-norm-persp-all/annotations_counts_IAA.csv"
df = pd.read_csv(iaa)
df.columns = ['claim_id','perspective_id','sup_raw','und_raw','ns_raw','sup','und','ns','agreement','annotator_count'] 

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4148 entries, 0 to 4147
Data columns (total 10 columns):
claim_id           4136 non-null float64
perspective_id     4136 non-null float64
sup_raw            4138 non-null object
und_raw            4144 non-null object
ns_raw             4148 non-null object
sup                4140 non-null float64
und                4140 non-null float64
ns                 4140 non-null float64
agreement          4136 non-null float64
annotator_count    4136 non-null float64
dtypes: float64(7), object(3)
memory usage: 324.1+ KB


In [63]:
sub_df = df[df['agreement'] > 0.43]
sub_df = sub_df[sub_df['annotator_count'] > 1]
sub_df['label'] = sub_df[['ns','sup','und']].idxmax(axis=1)
sub_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2493 entries, 9 to 2662
Data columns (total 11 columns):
claim_id           2493 non-null float64
perspective_id     2493 non-null float64
sup_raw            2493 non-null object
und_raw            2493 non-null object
ns_raw             2493 non-null object
sup                2493 non-null float64
und                2493 non-null float64
ns                 2493 non-null float64
agreement          2493 non-null float64
annotator_count    2493 non-null float64
label              2493 non-null object
dtypes: float64(7), object(4)
memory usage: 233.7+ KB


In [64]:
sup_und_df = sub_df[sub_df.label.isin(['sup', 'und'])]
sup_und_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2316 entries, 9 to 2662
Data columns (total 11 columns):
claim_id           2316 non-null float64
perspective_id     2316 non-null float64
sup_raw            2316 non-null object
und_raw            2316 non-null object
ns_raw             2316 non-null object
sup                2316 non-null float64
und                2316 non-null float64
ns                 2316 non-null float64
agreement          2316 non-null float64
annotator_count    2316 non-null float64
label              2316 non-null object
dtypes: float64(7), object(4)
memory usage: 217.1+ KB


In [65]:
out_path = "/home/squirrel/ccg-new/data/persp/db-101218-norm-persp-all/high_quality_index.csv"
sup_und_df.to_csv(out_path, index=False, columns=['claim_id', 'perspective_id', 'agreement','annotator_count','label'])

In [66]:
# Read claims and perspectives
claim_path = "/home/squirrel/ccg-new/data/persp/db-101218-norm-persp-all/claims.csv"
persp_path = "/home/squirrel/ccg-new/data/persp/db-101218-norm-persp-all/perspectives.csv"

c_df = pd.read_csv(claim_path)
p_df = pd.read_csv(persp_path)

cid_set = sup_und_df.claim_id.unique()
pid_set = sup_und_df.perspective_id.unique()

print(len(cid_set))
print(len(pid_set))

514
2316


In [67]:
c_df = c_df[c_df.id.isin(cid_set)]
c_df.info()

p_df = p_df[p_df.id.isin(pid_set)]
p_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 514 entries, 1 to 560
Data columns (total 3 columns):
id        514 non-null int64
source    514 non-null object
title     514 non-null object
dtypes: int64(1), object(2)
memory usage: 16.1+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2316 entries, 9 to 4132
Data columns (total 3 columns):
id        2316 non-null int64
source    2316 non-null object
title     2316 non-null object
dtypes: int64(1), object(2)
memory usage: 72.4+ KB


In [68]:
claim_outpath = "/home/squirrel/ccg-new/data/persp/db-101218-norm-persp-all/claims_high_quality.json"
persp_outpath = "/home/squirrel/ccg-new/data/persp/db-101218-norm-persp-all/perspective_high_quality.json"

c_df.to_json(claim_outpath, orient='records')
p_df.to_json(persp_outpath, orient='records')

In [69]:
anno_inpath = "/home/squirrel/ccg-new/data/persp/db-101218-norm-persp-all/high_quality_index.csv"
a_df = pd.read_csv(anno_inpath)

a_df['claim_id'] = a_df['claim_id'].astype(int)
a_df['perspective_id'] = a_df['perspective_id'].astype(int)
a_df['annotator_count'] = a_df['annotator_count'].astype(int)

a_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2316 entries, 0 to 2315
Data columns (total 5 columns):
claim_id           2316 non-null int64
perspective_id     2316 non-null int64
agreement          2316 non-null float64
annotator_count    2316 non-null int64
label              2316 non-null object
dtypes: float64(1), int64(3), object(1)
memory usage: 90.5+ KB


In [70]:
anno_outpath = "/home/squirrel/ccg-new/data/persp/db-101218-norm-persp-all/annotations_high_quality.json"

a_df.to_json(anno_outpath, orient='records')